In [22]:
import numpy as np
def feature_sign_search(A, y, gamma):
    tol = 1e-18
    # initialize x, theta and active_set
    x = np.zeros(A.shape[1])
    #print(x)

    theta = np.zeros(A.shape[1])
    active_set = set()
    
    # precompute AtA and Aty
    AtA = np.dot(A.T,A)
    #print(AtA)
    Aty = np.dot(A.T,y)
    #print(Aty)
    
    gradient = -2*Aty + 2*np.dot(AtA,x)
    #max_gradient = np.argmax(np.abs(gradient))
    
    zero_coeff = np.inf
    
    nonzero_coeff = 0
    
    # check the optimality conditions
    while not np.allclose(nonzero_coeff,0,tol) or zero_coeff > gamma:
        # condition (a) is true
        if np.allclose(nonzero_coeff,0):
            # condition (b) is not true, goto step 2
            # zero coefficients of x
            # print(theta,gradient)
            index = np.argmax(np.abs(gradient)*(theta==0))
            if gradient[index] > gamma:
                theta[index] = -1
                x[index] = 0
                active_set.add(index)
            elif gradient[index] < -gamma:
                theta[index] = 1
                x[index] = 0
                active_set.add(index)
            
            # condition (b) is true, exit from while loop
            if len(active_set) == 0: break
        # condition (a) is false, goto step 3
        #
        # feature-sign step
        sorted_active_set = sorted(active_set)
        indexArray = np.array(sorted_active_set)
        A_captA_cap = AtA[np.ix_(indexArray,indexArray)]
        A_capty = Aty[indexArray]
        x_cap = x[indexArray]
        theta_cap = theta[indexArray]
        
        # minimize x_cap ||y - A_cap x_cap||^2 + gamma*theta_capt*x_cap
        # singular matrix
        if np.linalg.det(A_captA_cap) == 0:
            x_cap_new = np.linalg.inv(A_captA_cap + 0.1*np.eye(A_captA_cap.shape[0])) * (A_capty - (gamma * theta_cap / 2))
        else:
            x_cap_new = np.linalg.inv(A_captA_cap) * (A_capty - (gamma * theta_cap / 2))
        theta_cap_new = np.sign(x_cap_new)

        sign_change = np.where(abs(theta_cap_new - theta_cap) == 2)[0]
        
        if len(sign_change) > 0:
            opt_x = x_cap_new
            opt_obj = np.dot(y.T,y) + (np.dot(x_cap_new, np.dot(A_captA_cap, x_cap_new))
                        - 2 * np.dot(x_cap_new, A_captA_cap) 
                        + gamma * abs(x_cap_new).sum())
            
            for i in sign_change:
                x1 = x_cap_new[i]
                x2 = x_cap[i]
                step = x2 / (x2 - x1)
                curr_x = x_cap - step * (x_cap - x_cap_new)
                obj = np.dot(y.T,y) + (np.dot(curr_x, np.dot(A_captA_cap, curr_x))
                        - 2 * np.dot(curr_x, A_captA_cap) 
                        + gamma * abs(curr_x).sum())
                if obj < opt_obj:
                    opt_obj = obj
                    opt_x = curr_x
        else:
            opt_x = x_cap_new
        # update x_cap (corresponding entries in x) to the point with the lowest objective value
        x[indexArray] = opt_x
        zeros = indexArray[np.abs(x[indexArray]) < tol]
        x_cap_new[zeros] = 0
        theta[indexArray] = np.sign(x[indexArray])
        active_set.difference_update(zeros)
        gradient = -2*Aty + 2*np.dot(AtA,x)
        zero_coeff = np.max(abs(gradient[theta==0]))
        nonzero_coeff = np.max(abs(gradient[theta!=0] + gamma * theta[theta!=0]))
    return x
        
                

In [24]:
a = np.array([[1,2,3],[4,5,6]])
y = np.array([1,2])
g = 0.2

feature_sign_search(a,y,g)

array([0.        , 0.        , 0.33111111])